In [1]:
import pandas as pd
import json
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
#@title Service Account Json

# Carregue o arquivo JSON com suas credenciais do Google Cloud
credentials = service_account.Credentials.from_service_account_info(
    json.loads(open("projeto-noticias-394817-324ae2c7db36.json", "r").read())
)

In [3]:
#@title Tratamento do arquivo Json
# Carregue o arquivo JSON em uma estrutura de dados Python
with open('notices_globo.json', 'r') as json_file:
    data = json.load(json_file)

In [4]:
df = pd.DataFrame(data)

In [5]:
def limpar_string(valor_coluna):
    """
    Função realiza o replace nos valores "n\" que vieram junto com a raspagem
    Os valores de 'Date' também vieram com o horário de publicação, a função
    realiza um split nessas informações. Ex: '19.jul.2023 às 19h46'
    """
    if isinstance(valor_coluna, str):
        if 'às' in valor_coluna:
            parte_data, parte_hora = valor_coluna.split('às')
            parte_data = parte_data.strip()
            # Reconstrói a string sem a parte da hora
            string_limpa = f"{parte_data}"
        else:
            # Se não houver 'parte da hora', mantém a string original
            string_limpa = valor_coluna
        return string_limpa.replace('\n', '').strip()
    else:
        return valor_coluna


In [6]:
# Aplica a função limpar_string nas colunas 'title', 'date' e 'newspaper_section'
df['title'] = df['title'].apply(limpar_string)
df['date'] = df['date'].apply(limpar_string)
df['newspaper_section'] = df['newspaper_section'].apply(limpar_string)
df['subtitle'] = df['subtitle'].apply(limpar_string)

In [7]:
def converter_format_date(date_str):
    """
    Função altera valores de meses abreviados na coluna 'Date' para
    os números respectivos de cada mês e também converte o formato.
    """
    try:
        meses_str = [
            'jan', 'fev', 'mar', 'abr', 'mai', 'jun',
            'jul', 'ago', 'set', 'out', 'nov', 'dez'
        ]

        date_obj = datetime.strptime(date_str, '%d.%b.%Y')
        meses_str = date_str.split('.')[1].lower()
        numero_mes = meses_str.index(meses_str)
        date = date_obj.replace(month=numero_mes).strftime('%Y-%m-%d')
        return date
    except:
        return date_str

In [8]:
# Aplica a função de conversão na coluna 'date'
df['date'] = df['date'].apply(converter_format_date)

In [9]:
df.head()

,link,title,date,newspaper_section,subtitle
0,//www.globo.com/busca/click?q=Seguran%C3%A7a+d...,Chefe de segurança da informação do Twitter de...,10/11/2022 17h37,epocanegocios,Lea Kissner anuniou a saída do TwitterReproduç...
1,//www.globo.com/busca/click?q=Seguran%C3%A7a+d...,Segurança da informação: o que é e como se tor...,23/09/2022 10h01,None,Realizar a proteção de dados na internet é um ...
2,//www.globo.com/busca/click?q=Seguran%C3%A7a+d...,A segurança da informação em tempos de home of...,09/06/2022 17h11,Rede Globo,".... Entretanto, com aumento de troca de"
3,//www.globo.com/busca/click?q=Seguran%C3%A7a+d...,FICE 2021: Para aumentar segurança da informaç...,22/10/2021 15h47,Pequenas Empresas e Grandes Negócios,... de
4,//www.globo.com/busca/click?q=Seguran%C3%A7a+d...,FICE 2021: Para aumentar segurança da informaç...,14/10/2021 21h04,Época Negócios,"... usada pela holandesa Jaya Baloo, diretora ..."


In [10]:
#@title Parte do código onde os valores são lançados no Big Query

# Substitua project_id pelo ID do seu projeto no Google Cloud
project_id = 'projeto-noticias-394817'

In [11]:
# Substitua dataset_id pelo ID do conjunto de dados que você deseja usar
dataset_id = 'noticias'


In [12]:
# Substitua table_id pelo ID da tabela que você deseja usar
table_id = 'globo'

In [13]:
# Envia o DataFrame para o BigQuery
client = bigquery.Client(project=project_id, credentials=credentials)
table_id = f"{project_id}.{dataset_id}.{table_id}"

In [14]:
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("link", "STRING"),
        bigquery.SchemaField("title", "STRING"),
        bigquery.SchemaField("date", "STRING"),
        bigquery.SchemaField("newspaper_section", "STRING"),
        bigquery.SchemaField("subtitle", "STRING"),
    ],
    write_disposition="WRITE_TRUNCATE",  # Modo de escrita: WRITE_TRUNCATE irá substituir a tabela existente
)

In [15]:
# Carrega os dados para o BigQuery
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

In [16]:
job.result()  # Aguarda a conclusão do job
print(f"Os dados foram enviados para a tabela: {table_id}")

Os dados foram enviados para a tabela: projeto-noticias-394817.noticias.globo
